In [860]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

# Data Description

* Population
* GDP
* DCT - $CO_{2}$ Emissions – Total - `CDI = CO2 Intensity Current-Year Score`
* DPT - $CO_{2}$ emissions/kWh elect. & heat - `CEH = CO2 Emissions per kWh`
* DMT - $CH_{4}$ emissions - `CHI = $CH_{4}$ Intensity Current-Year Score`
* DNT - $N_{2}O$ emissions - `NOI = $N_{2}O$ Intensity Current-Year Score`
* DBT - Black Carbon emissions - `BCI = Black Carbon Current-Year Score`

In [861]:
pop = pd.read_csv('pop_corr.csv')
gdp = pd.read_csv('gdp_corr.csv')
dpt = pd.read_csv('dpt_corr.csv')
dnt = pd.read_csv('dnt_corr.csv')
dbt = pd.read_csv('dbt_corr.csv')
dct = pd.read_csv('dct_corr.csv')
dmt = pd.read_csv('dmt_corr.csv')

In [862]:
countries = list(pop['country'].unique())

In [863]:
pop.drop(columns=['Unnamed: 0'],inplace=True)
gdp.drop(columns=['Unnamed: 0'],inplace=True)
dpt.drop(columns=['Unnamed: 0'],inplace=True)
dnt.drop(columns=['Unnamed: 0'],inplace=True)
dbt.drop(columns=['Unnamed: 0'],inplace=True)
dct.drop(columns=['Unnamed: 0'],inplace=True)
dmt.drop(columns=['Unnamed: 0'],inplace=True)

In [864]:
print('Population has {0} missing values'.format(pop.isna().sum()[2]))
print('GDP has {0} missing values'.format(gdp.isna().sum()[2]))
print('DPT has {0} missing values'.format(dpt.isna().sum()[2]))
print('DNT has {0} missing values'.format(dnt.isna().sum()[2]))
print('DBT has {0} missing values'.format(dbt.isna().sum()[2]))
print('DCT has {0} missing values'.format(dct.isna().sum()[2]))
print('DMT has {0} missing values'.format(dmt.isna().sum()[2]))

Population has 361 missing values
GDP has 804 missing values
DPT has 937 missing values
DNT has 900 missing values
DBT has 689 missing values
DCT has 906 missing values
DMT has 900 missing values


# Fisrt we need to separate the data per country

## Then we have to handle with the missing values

## For some variables we have to use time series to fill the year block (1997-2014)

### We have 18 years history. So the length of each country should be 18.

In [865]:
def col_country(df):
    lst = []
    for country in countries:
        test = df[df['country']==str(country)]
        lst.append(test)
    return lst

In [866]:
pop_count = col_country(pop)
gdp_count = col_country(gdp)
dpt_count = col_country(dpt)
dnt_count = col_country(dnt)
dbt_count = col_country(dbt)
dct_count = col_country(dct)
dmt_count = col_country(dmt)

In [867]:
var_list = [pop_count,gdp_count,dpt_count,dnt_count,dbt_count,dct_count,dmt_count]
variables = ['pop','GDP','dpt','dnt','dbt','dct','dmt']

In [868]:
all_data = [] # it's has to be 236 elements 
for i in range(len(countries)):
    df1 = pd.merge(var_list[0][i],var_list[1][i],how='outer')
    df2 = pd.merge(df1,var_list[2][i],how='outer')
    df3 = pd.merge(df2,var_list[3][i],how='outer')
    df4 = pd.merge(df3,var_list[5][i],how='outer')
    df5 = pd.merge(df4,var_list[6][i],how='outer')
    df6 = pd.merge(df5,var_list[4][i],how='outer')
    all_data.append(df6)

In [869]:
len(all_data) # it has to be 236

236

In [870]:
c1 = [] # country
c2 = [] # population
c3 = [] # GDP
c4 = [] # DPT
c5 = [] # DNT
c6 = [] # DCT
c7 = [] # DMT
c8 = [] # DBT

for i in range(len(countries)):
    c1.append(countries[i])
    c2.append(all_data[i].isna().sum()[2])
    c3.append(all_data[i].isna().sum()[3])
    c4.append(all_data[i].isna().sum()[4])
    c5.append(all_data[i].isna().sum()[5])
    c6.append(all_data[i].isna().sum()[6])
    c7.append(all_data[i].isna().sum()[7])
    c8.append(all_data[i].isna().sum()[8])

In [871]:
df = np.array([c2,c3,c4,c5,c6,c7,c8])
data = pd.DataFrame(df)

In [872]:
data['variables'] = ['pop','GDP','dpt','dnt','dct','dmt','dbt']
data.set_index('variables',inplace=True)

In [873]:
data.columns = c1

In [874]:
data = data.T.reset_index()

In [875]:
data.shape

(236, 8)

In [876]:
data.head()

variables,index,pop,GDP,dpt,dnt,dct,dmt,dbt
0,Afghanistan,0,5,18,5,5,5,9
1,Albania,0,0,8,0,0,0,4
2,Algeria,0,0,8,0,0,0,4
3,American Samoa,0,18,18,18,18,18,18
4,Andorra,0,18,18,18,18,18,18


# Handle the missing data per country

## `all_data` is a list with 236 elements, each one is a country dataframe

## `data` is a dataframe with the amount of missing values for each country for all features

In [877]:
data.head(10)

variables,index,pop,GDP,dpt,dnt,dct,dmt,dbt
0,Afghanistan,0,5,18,5,5,5,9
1,Albania,0,0,8,0,0,0,4
2,Algeria,0,0,8,0,0,0,4
3,American Samoa,0,18,18,18,18,18,18
4,Andorra,0,18,18,18,18,18,18
5,Angola,0,0,8,0,0,0,4
6,Anguilla,18,18,18,18,18,18,18
7,Antigua and Barbuda,0,0,18,0,0,0,4
8,Argentina,0,0,8,0,0,0,4
9,Armenia,0,0,8,0,0,0,4


### The sum of all possibles NaN values is 126

In [878]:
data['tot'] = data['pop']+data['GDP']+data['dpt']+data['dnt']+data['dct']+data['dmt']+data['dbt']

### If a country has more than 63 (50%) of missing values, we are going to drop this country

In [879]:
data['drop'] = data['tot'].apply(lambda x: True if x>=63 else False) # 50% == 63

In [880]:
data.groupby('drop').agg({'index':'count'})

,index
drop,
False,187
True,49


In [881]:
# make a list with the country that we have to drop in all_data
drop_data = data[data['drop']==True]
drop_ = list(drop_data['index'])

In [882]:
len(drop_)

49

In [883]:
data_filtered = data[data['drop']==False]
data_filtered.drop(columns=['tot','drop'],inplace=True)

In [884]:
len(data_filtered)

187

In [885]:
data_filtered.head(30)

variables,index,pop,GDP,dpt,dnt,dct,dmt,dbt
0,Afghanistan,0,5,18,5,5,5,9
1,Albania,0,0,8,0,0,0,4
2,Algeria,0,0,8,0,0,0,4
5,Angola,0,0,8,0,0,0,4
7,Antigua and Barbuda,0,0,18,0,0,0,4
8,Argentina,0,0,8,0,0,0,4
9,Armenia,0,0,8,0,0,0,4
11,Australia,0,0,8,0,0,0,4
12,Austria,0,0,8,0,0,0,4
13,Azerbaijan,0,0,8,0,0,0,4


In [886]:
drop_30 = ['Afghanistan','Antigua and Barbuda','Bahamas','Barbados','Belize','Bhutan','Burkina Faso','Burundi']

In [887]:
data_filtered[30:60]

variables,index,pop,GDP,dpt,dnt,dct,dmt,dbt
36,Cape Verde,0,0,18,0,0,0,4
38,Central African Republic,0,0,18,0,0,0,4
39,Chad,0,0,18,0,0,0,4
40,Chile,0,0,8,0,0,0,4
41,China,0,0,8,0,0,0,4
42,Colombia,0,0,8,0,0,0,4
43,Comoros,0,0,18,0,0,0,4
45,Costa Rica,0,0,8,0,0,0,4
46,Cote d'Ivoire,0,0,8,0,0,0,4
47,Croatia,0,0,8,0,0,0,4


In [888]:
drop_60 = ['Cape Verde','Central African Republic','Chad','Comoros','Djibouti','Dominica','Equatorial Guinea','Fiji','Gambia']

In [889]:
data_filtered[60:90]

variables,index,pop,GDP,dpt,dnt,dct,dmt,dbt
73,Georgia,0,0,8,0,0,0,4
74,Germany,0,0,8,0,0,0,4
75,Ghana,0,0,8,0,0,0,4
77,Greece,0,0,8,0,0,0,4
79,Grenada,0,0,18,0,0,0,4
82,Guatemala,0,0,8,0,0,0,4
83,Guinea,0,0,18,0,0,0,4
84,Guinea-Bissau,0,0,18,0,0,0,4
85,Guyana,0,0,18,0,0,0,4
86,Haiti,0,0,8,0,0,0,4


In [890]:
drop_90 = ['Grenada','Guinea','Guinea-Bissau','Guyana','Kiribati','Laos']

In [891]:
data_filtered[90:120]

variables,index,pop,GDP,dpt,dnt,dct,dmt,dbt
111,Lebanon,0,0,8,0,0,0,4
112,Lesotho,0,0,18,0,0,0,4
113,Liberia,0,0,18,0,0,0,4
114,Libya,0,0,8,0,0,0,4
116,Lithuania,0,0,8,0,0,0,4
117,Luxembourg,0,0,8,0,0,0,4
119,Macedonia,0,0,8,0,0,0,4
120,Madagascar,0,0,18,0,0,0,4
121,Malawi,0,0,18,0,0,0,4
122,Malaysia,0,0,8,0,0,0,4


In [892]:
drop_120 = ['Lesotho','Liberia','Madagascar','Malawi','Maldives','Mali','Marshall Islands','Mauritania','Micronesia']

In [893]:
data_filtered[120:150]

variables,index,pop,GDP,dpt,dnt,dct,dmt,dbt
150,Nigeria,0,0,8,0,0,0,4
155,Norway,0,0,8,0,0,0,4
156,Oman,0,0,8,0,0,0,4
157,Pakistan,0,0,8,0,0,0,4
158,Palau,0,0,18,0,0,0,18
159,Panama,0,0,8,0,0,0,4
160,Papua New Guinea,0,0,18,0,0,0,4
161,Paraguay,0,0,8,0,0,0,4
162,Peru,0,0,8,0,0,0,4
163,Philippines,0,0,8,0,0,0,4


In [894]:
drop_150 = ['Palau','Papua New Guinea','Rwanda','Saint Kitts and Nevis','Saint Lucia','Saint Vincent and the Grenadines','Samoa','Sao Tome and Principe','Seychelles','Sierra Leone']

In [895]:
data_filtered[150:]

variables,index,pop,GDP,dpt,dnt,dct,dmt,dbt
191,Solomon Islands,0,0,18,0,0,0,4
192,Somalia,0,18,18,0,0,0,4
193,South Africa,0,0,8,0,0,0,4
194,South Korea,0,0,8,0,0,0,4
196,Spain,0,0,8,0,0,0,4
197,Sri Lanka,0,0,8,0,0,0,4
199,Sudan,0,0,8,0,0,0,4
200,Suriname,0,0,8,0,0,0,4
202,Swaziland,0,0,18,0,0,0,4
203,Sweden,0,0,8,0,0,0,4


In [896]:
drop_187 = ['Solomon Islands','Somalia','Swaziland','Syria','Timor-Leste','Tonga','Tuvalu','Uganda','Vanuatu']

### Join all countries in list to drop

In [897]:
final_drop = drop_ + drop_30 + drop_60 + drop_90 + drop_120 + drop_150 + drop_187

In [898]:
len(final_drop)

100

In [899]:
li=[]
for country in final_drop:
    ind = countries.index(str(country))
    li.append(ind)

#ind_to_remove=list(set(li))

In [900]:
final = []
for ind in range(len(all_data)):
    if ind not in li:
        final.append(all_data[ind])

In [901]:
len(final)

136

## We have information about 136 countries

In [902]:
def list_to_df(lst):
    df = lst[0]
    for item in range(1,len(lst)):
        df = df.append(lst[item])
    return df

In [903]:
data_complete = list_to_df(final)

In [904]:
data_complete.isna().sum()

country          0
year             0
population       0
GPD              0
DPT           1088
DNT              4
DCT             12
DMT              4
DBT            547
dtype: int64

# `DBT` Column

# Generate data for 2011-2014

## We expect 552 NaN in * DBT * but we have 555, so that means we have 3 NaN n the years we collected information

In [905]:
from statsmodels.tsa.ar_model import AR
import datetime

In [906]:
for country in range(0,len(final)):
    if final[country].isna().sum()[-1] == 4:
        df = final[country]
        df.index = pd.to_datetime(df['year'],format='%Y').dt.year
        features = df['DBT']
        ff = features[:-4]
        model = AR(ff)
        model_fit = model.fit(maxlag=3)
        d_f = model_fit.predict(start=len(ff), end=len(ff)+3, dynamic=False)
        d_f_final = ff.to_list() + d_f.to_list()
        df['DBT'] = d_f_final
    elif final[country].isna().sum()[-1] ==0:
        continue
    else:
        df = final[country]
        features = df['DBT'].fillna(df['DBT'].mean())
        ff = features[:-4]
        model = AR(ff)
        model_fit = model.fit(maxlag=3)
        d_f = model_fit.predict(start=len(ff), end=len(ff)+3, dynamic=False)
        d_f_final = ff.to_list() + d_f.to_list()
        df['DBT'] = d_f_final

l be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
C:\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsuppor

# `DPT` Column

# Generate data for 1997-2004

### The time series here has to be inverted

In [907]:
final.pop(0)
len(final)

135

### I decide to remove `Albania` because the values for DPT was to different from the pdf and has 3 or 4 zeros

In [908]:
data_complete = list_to_df(final)
data_complete.isna().sum()

country          0
year             0
population       0
GPD              0
DPT           1080
DNT              4
DCT             12
DMT              4
DBT              0
dtype: int64

In [917]:
data_complete.describe()

,year,population,GPD,DPT,DNT,DCT,DMT,DBT
count,2430.000000,2.430000e+03,2.430000e+03,1350.000000,2426.000000,2418.000000,2426.000000,2430.000000
mean,2005.500000,4.657536e+07,5.749026e+11,444.998245,-23.872471,-22.122728,-22.941655,-23.809777
std,5.189195,1.523057e+08,1.645009e+12,281.960631,1.314099,0.614153,1.225970,1.343224
min,1997.000000,2.711280e+05,4.594740e+09,0.000000,-27.593931,-24.129480,-26.279139,-28.478197
25%,2001.000000,4.384600e+06,3.258659e+10,247.733325,-24.715302,-22.516180,-23.847078,-24.803556
50%,2005.500000,1.042038e+07,1.106673e+11,436.640900,-23.970881,-22.141306,-22.985642,-23.980115
75%,2010.000000,3.374609e+07,3.945377e+11,645.052000,-23.039876,-21.769983,-22.025259,-22.971090
max,2014.000000,1.364270e+09,1.740624e+13,3297.680000,-20.072658,-20.026602,-19.371484,-20.284348


### We expect 1080 NaN in * DPT *

In [918]:
for country in range(0,len(final)):
    if final[country].isna().sum()[4] == 8:
        test = final[country]
        #test['DPT'] = (test['DPT'].fillna(test['DPT'].mean())).to_list()
        dpt_arr = test['DPT'].to_list()
        year_arr = test['year'].to_list()
        dpt_inv = []
        for i in reversed(dpt_arr):
             dpt_inv.append(i)
        time = year_arr[8:]
        y = dpt_inv[:10]
        df = pd.DataFrame(y,time,columns=['DPT']).reset_index()
        df.index = pd.to_datetime(df['index'],format='%Y').dt.year
        features = df['DPT']
        model = AR(features)
        model_fit = model.fit(maxlag=2)
        d_f = model_fit.predict(start=len(features), end=len(features)+7, dynamic=False)
        d_f_final = features.to_list() + d_f.to_list()
        df_final_inv = []
        for i in reversed(d_f_final):
            df_final_inv.append(i)
        final[country]['DPT'] = df_final_inv
    else:
        continue

In [930]:
data_complete[data_complete['country']=='Iceland']

,country,year,population,GPD,DPT,DNT,DCT,DMT,DBT
1997,Iceland,1997,271128.0,8.264269e+09,NaN,-24.166223,-22.064499,-23.631706,-24.624482
1998,Iceland,1998,274047.0,8.806997e+09,NaN,-24.190907,-22.129341,-23.685497,-24.756989
1999,Iceland,1999,277381.0,9.159613e+09,NaN,-24.194679,-22.188565,-23.726203,-24.843174
2000,Iceland,2000,281205.0,9.588330e+09,NaN,-24.253962,-22.180341,-23.790400,-24.817740
2001,Iceland,2001,284968.0,9.955922e+09,NaN,-24.384069,-22.250266,-23.834725,-24.822879
2002,Iceland,2002,287523.0,9.987254e+09,NaN,-24.339867,-22.217964,-23.855607,-24.822645
2003,Iceland,2003,289521.0,1.023096e+10,NaN,-24.378090,-22.244905,-23.885805,-24.945771
2004,Iceland,2004,292074.0,1.105964e+10,NaN,-24.468052,-22.284443,-23.975307,-25.051288
2005,Iceland,2005,296734.0,1.180112e+10,0.449,-24.521709,-22.360313,-24.033027,-25.227709
2006,Iceland,2006,303782.0,1.239126e+10,0.411,-24.497584,-22.390270,-24.041945,-25.060110


In [928]:
final[50]

,country,year,population,GPD,DPT,DNT,DCT,DMT,DBT
year,,,,,,,,,
1997,Iceland,1997,271128.0,8.264269e+09,1888.792999,-24.166223,-22.064499,-23.631706,-24.624482
1998,Iceland,1998,274047.0,8.806997e+09,-224.786605,-24.190907,-22.129341,-23.685497,-24.756989
1999,Iceland,1999,277381.0,9.159613e+09,-230.063485,-24.194679,-22.188565,-23.726203,-24.843174
2000,Iceland,2000,281205.0,9.588330e+09,134.741130,-24.253962,-22.180341,-23.790400,-24.817740
2001,Iceland,2001,284968.0,9.955922e+09,38.634965,-24.384069,-22.250266,-23.834725,-24.822879
2002,Iceland,2002,287523.0,9.987254e+09,-18.644028,-24.339867,-22.217964,-23.855607,-24.822645
2003,Iceland,2003,289521.0,1.023096e+10,5.670480,-24.378090,-22.244905,-23.885805,-24.945771
2004,Iceland,2004,292074.0,1.105964e+10,7.464946,-24.468052,-22.284443,-23.975307,-25.051288
2005,Iceland,2005,296734.0,1.180112e+10,0.449000,-24.521709,-22.360313,-24.033027,-25.227709


In [919]:
dpt_complete = list_to_df(final)
dpt_complete.isna().sum()

country       0
year          0
population    0
GPD           0
DPT           0
DNT           0
DCT           0
DMT           0
DBT           0
dtype: int64

In [920]:
for country in range(0,len(final)):
    if final[country].isna().sum()[-1] == 4:
        df = pd.DataFrame(final[country])
        df.index = pd.to_datetime(df['year'],format='%Y').dt.year
        features = df['DBT']
        ff = features[8:]
        model = AR(ff)
        model_fit = model.fit(maxlag=3)
        d_f = model_fit.predict(start=len(ff), end=len(ff)+3, dynamic=False)
        d_f_final = ff.to_list() + d_f.to_list()
        df['DBT'] = d_f_final
    elif final[country].isna().sum()[-1] ==0:
        continue
    else:
        df = pd.DataFrame(final[country])
        features = df['DBT'].fillna(df['DBT'].mean())
        ff = features[:-4]
        model = AR(ff)
        model_fit = model.fit(maxlag=5)
        d_f = model_fit.predict(start=len(ff), end=len(ff)+3, dynamic=False)
        d_f_final = ff.to_list() + d_f.to_list()
        df['DBT'] = d_f_final

# Others columns

In [921]:
for country in range(0,len(final)):
    df = pd.DataFrame(final[country])
    if final[country].isna().sum()[-2] == 0:
        continue
    else:
        df['DMT'].fillna(df['DMT'].mean(),inplace=True)

In [922]:
for country in range(0,len(final)):
    df = pd.DataFrame(final[country])
    if final[country].isna().sum()[-3] == 0:
        continue
    else:
        df['DCT'].fillna(df['DCT'].mean(),inplace=True)

In [923]:
for country in range(0,len(final)):
    df = pd.DataFrame(final[country])
    if final[country].isna().sum()[-4] == 0:
        continue
    else:
        df['DNT'].fillna(df['DNT'].mean(),inplace=True)

# Join all Dataframes

In [924]:
data_clean = list_to_df(final)
data_clean.isna().sum()

country       0
year          0
population    0
GPD           0
DPT           0
DNT           0
DCT           0
DMT           0
DBT           0
dtype: int64

In [925]:
data_clean.shape

(2430, 9)

In [931]:
data_clean.drop(columns=['DPT'],inplace=True)

In [932]:
data_clean.describe()

,year,population,GPD,DNT,DCT,DMT,DBT
count,2430.000000,2.430000e+03,2.430000e+03,2430.000000,2430.000000,2430.000000,2430.000000
mean,2005.500000,4.657536e+07,5.749026e+11,-23.875993,-22.121880,-22.944242,-23.809777
std,5.189195,1.523057e+08,1.645009e+12,1.315913,0.612852,1.226631,1.343224
min,1997.000000,2.711280e+05,4.594740e+09,-27.593931,-24.129480,-26.279139,-28.478197
25%,2001.000000,4.384600e+06,3.258659e+10,-24.721263,-22.512780,-23.850064,-24.803556
50%,2005.500000,1.042038e+07,1.106673e+11,-23.974644,-22.139406,-22.988277,-23.980115
75%,2010.000000,3.374609e+07,3.945377e+11,-23.041925,-21.772908,-22.031655,-22.971090
max,2014.000000,1.364270e+09,1.740624e+13,-20.072658,-20.026602,-19.371484,-20.284348


# Export to csv and excel

In [934]:
data_clean.to_excel('data_clean.xlsx')
data_clean.to_csv('data_clean.csv')